# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room Part #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [6]:
!pip install -qU ragas

In [37]:
%conda update bottleneck


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
/ DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/osx-64/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 200 None
- DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/osx-64/current_repodata.json HTTP/1.1" 200 None
done
Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::numba==0.57.0=py311hdb55bb0_0
  - defaults/noarch::tifffile==2021.7.2=pyhd

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [4]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [1]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [7]:
!git clone https://github.com/AI-Maker-Space/DataRepository

Cloning into 'DataRepository'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 68 (delta 18), reused 28 (delta 8), pack-reused 8
Receiving objects: 100% (68/68), 69.00 MiB | 1.92 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/tswift_fued.pdf",
)

documents = loader.load()

In [3]:
documents[0].metadata

{'source': 'DataRepository/tswift_fued.pdf',
 'file_path': 'DataRepository/tswift_fued.pdf',
 'page': 0,
 'total_pages': 22,
 'format': 'PDF 1.4',
 'title': "A Timeline of Taylor Swift and Kim Kardashian's Feud",
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
 'producer': 'Skia/PDF m123',
 'creationDate': "D:20240423220523+00'00'",
 'modDate': "D:20240423220523+00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)


Let's confirm we've split our document.

In [5]:
len(documents)

177

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [7]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="Taylor Swift - Fued - ADA",
)

qdrant_vector_store

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about FAISS!


- FAISS: Facebook AI similarity search
- HNSW: Hierarchical Navigable Small World
- Euclidian distance, cosine similarity, dot product

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [8]:
retriever = qdrant_vector_store.as_retriever()

retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [10]:
retrieved_documents = retriever.invoke("Who is Taylor Swift fueding with?")


In [11]:
for doc in retrieved_documents:
  print(doc)

page_content="4/23/24, 6:05 PM\nA Timeline of Taylor Swift and Kim Kardashian's Feud\nhttps://people.com/taylor-swift-and-kim-kardashian-feud-timeline-8412119\n19/22\nMUSIC" metadata={'source': 'DataRepository/tswift_fued.pdf', 'file_path': 'DataRepository/tswift_fued.pdf', 'page': 18, 'total_pages': 22, 'format': 'PDF 1.4', 'title': "A Timeline of Taylor Swift and Kim Kardashian's Feud", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'creationDate': "D:20240423220523+00'00'", 'modDate': "D:20240423220523+00'00'", 'trapped': '', '_id': '9eb7ae3774f3463798c54a42096a7799', '_collection_name': 'Taylor Swift - Fued - ADA'}
page_content="4/23/24, 6:05 PM\nA Timeline of Taylor Swift and Kim Kardashian's Feud\nhttps://people.com/taylor-swift-and-kim-kardashian-feud-timeline-8412119\n21/22\nMUSIC" metadata={'source': 'DataRepository/tswift_fue

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [12]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [13]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [14]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [15]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

Let's test it out!

In [16]:
question = "Who is Taylor Swift fueding with?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Kim Kardashian


In [17]:
question = "Why are they fueding?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content="discussed the longstanding feud between West and Swift in great detail, as\nwell as West's controversial rant during his appearance on Saturday Night", metadata={'source': 'DataRepository/tswift_fued.pdf', 'file_path': 'DataRepository/tswift_fued.pdf', 'page': 6, 'total_pages': 22, 'format': 'PDF 1.4', 'title': "A Timeline of Taylor Swift and Kim Kardashian's Feud", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'creationDate': "D:20240423220523+00'00'", 'modDate': "D:20240423220523+00'00'", 'trapped': '', '_id': '2654a02fc9c54c408c7b0c46f89f2f24', '_collection_name': 'Taylor Swift - Fued - ADA'}), Document(page_content='That Beef!" segment. When asked about if her feud with Swift over the\nleaked phone calls was still ongoing, she said she was "over it."', metadata={'source': 'DataRepository/tswif

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room Part #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [18]:
loader = PyMuPDFLoader(
    "DataRepository/tswift_fued.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

In [19]:
len(eval_documents)

56

> NOTE: This cell will take ~5-6min. to generate. If you run into any rate-limit issues - please use GPT-3.5-Turbo as your `critic_llm`. If you see any fields marked `nan` - this is product of rate-limiting issues, and you can safely ignore them for now.

In [27]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo")
# critic_llm = ChatOpenAI(model="gpt-4-turbo")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

embedding nodes:   0%|          | 0/112 [00:00<?, ?it/s]

KeyboardInterrupt: 

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

A: sets the question type distribution of the test data:
- 25% simple quesiton
- 25% reasoning: question requires additional reasoning to answer effectively
- 50% multi-context: requires multiple pieces of context not just a simple one

Let's look at the output and see what we can learn about it!

In [211]:
testset.test_data[0]

DataRow(question='What was the response on the internet when #KimExposedTaylorParty started trending?', contexts=["claimed that Swift knew of West's plans to release the track as she\ninfamously leaked an edited phone call between West and Swift discussing\nthe song before its release. The Internet immediately seemed to side with\nthe reality star, and #KimExposedTaylorParty started trending.\nThe following year, Swift responded to the drama with her sixth studio\nalbum Reputation, which was dominated by snake imagery as she\nreclaimed the snake emoji posted in her Instagram comments section at the\ntime.\xa0\nMost recently, Swift reflected on the feud yet again with two new tracks"], ground_truth='The Internet immediately seemed to side with the reality star', evolution_type='simple', metadata=[{'source': 'DataRepository/tswift_fued.pdf', 'file_path': 'DataRepository/tswift_fued.pdf', 'page': 1, 'total_pages': 22, 'format': 'PDF 1.4', 'title': "A Timeline of Taylor Swift and Kim Karda

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [219]:
test_df = testset.to_pandas()

In [220]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What was the response on the internet when #Ki...,[claimed that Swift knew of West's plans to re...,The Internet immediately seemed to side with t...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
1,What is the significance of Taylor Swift's vau...,[By Kelsie Gibson\nMUSIC\nTaylor Swift Referen...,nan,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
2,How did Taylor Swift use Instagram videos to r...,"[the release of her sixth studio album, Reputa...",Taylor Swift used Instagram videos depicting a...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
3,What organization did Taylor Swift mention as ...,"[leaked call.\n""The World Health Organization ...",The World Health Organization and Feeding America,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
4,What was the significance of Kim Kardashian re...,[motivation to flip … maybe she took it the wr...,nan,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
5,What is the timeline of Taylor Swift and Kim K...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",nan,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
6,What does Taylor Swift hope to receive from pe...,"[screaming fans,"" she wrote. ""It's the Stadium...",It would be nice if we could get an apology fr...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
7,What was the response on the internet when #Ki...,[claimed that Swift knew of West's plans to re...,The Internet immediately seemed to side with t...,simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
8,"What impact did the feud between Taylor Swift,...","[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...","The feud between Taylor Swift, Kanye West, and...",simple,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True
9,How did Taylor Swift address the controversy w...,[claimed that Swift knew of West's plans to re...,Swift addressed the controversy with Kanye Wes...,multi_context,"[{'source': 'DataRepository/tswift_fued.pdf', ...",True


In [221]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [222]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [223]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [224]:
response_dataset[0]

{'question': 'What was the response on the internet when #KimExposedTaylorParty started trending?',
 'answer': 'The Internet immediately seemed to side with the reality star.',
 'contexts': ['actual lyrics, \'I made that bitch famous.\'"\nThe Internet immediately seemed to side with the reality star, and\n#KimExposedTaylorParty started trending.\nJuly 17\n, 2016: Taylor Swift defends herself',
  'the reality star, and #KimExposedTaylorParty started trending.\nThe following year, Swift responded to the drama with her sixth studio\nalbum Reputation, which was dominated by snake imagery as she',
  'then Kim posts it on the Internet."\nAugust 27\n, 2017: Taylor Swift references\nKanye West and Kim Kardashian drama in\n“Look What You Made Me Do” music video\xa0\nTaylor Swift - Look What You Made Me Do',
  'asked her to release the song on her Twitter account, which she declined to\ndo."\nJuly 17\n, 2016: Kim Kardashian explains why\nshe defended Kanye West against Taylor\nSwift'],
 'ground_

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [225]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [226]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/90 [00:00<?, ?it/s]

In [227]:
results

{'faithfulness': 0.9643, 'answer_relevancy': 0.6933, 'context_recall': 0.6296, 'context_precision': 0.6852, 'answer_correctness': 0.5710}

In [228]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What was the response on the internet when #Ki...,The Internet immediately seemed to side with t...,"[actual lyrics, 'I made that bitch famous.'""\n...",The Internet immediately seemed to side with t...,1.0,0.808814,1.000000,1.000000,0.997215
1,What is the significance of Taylor Swift's vau...,I don't know.,[Vault Track 'Is It Over Now?': Listen\nBy Dan...,nan,NaN,0.000000,0.000000,0.000000,0.198202
2,How did Taylor Swift use Instagram videos to r...,Taylor Swift used a series of mysterious Insta...,[with a series of mysterious Instagram videos ...,Taylor Swift used Instagram videos depicting a...,1.0,0.964812,1.000000,0.916667,0.616807
3,What organization did Taylor Swift mention as ...,The World Health Organization and Feeding Amer...,"[me in donating during this crisis,"" continued...",The World Health Organization and Feeding America,1.0,0.954495,1.000000,1.000000,0.997151
4,What was the significance of Kim Kardashian re...,The significance was to show that Taylor Swift...,"[going to take it.""\nJuly 17\n, 2016: Kim Kard...",nan,1.0,0.932975,0.333333,1.000000,0.178299
5,What is the timeline of Taylor Swift and Kim K...,I don't know.,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",nan,NaN,0.000000,0.000000,0.000000,0.198202
6,What does Taylor Swift hope to receive from pe...,An apology.,[to a troll's hateful Instagram comment with '...,It would be nice if we could get an apology fr...,1.0,0.746619,1.000000,1.000000,0.963934
7,What was the response on the internet when #Ki...,The Internet immediately seemed to side with t...,"[actual lyrics, 'I made that bitch famous.'""\n...",The Internet immediately seemed to side with t...,1.0,0.803178,1.000000,1.000000,0.997215
8,"What impact did the feud between Taylor Swift,...",The feud with Kanye West and Kim Kardashian fe...,[19/22\nMUSIC\nTaylor Swift Says Feud with Kan...,"The feud between Taylor Swift, Kanye West, and...",1.0,0.929783,1.000000,1.000000,0.612551
9,How did Taylor Swift address the controversy w...,I don't know.,[she defended Kanye West against Taylor\nSwift...,Swift addressed the controversy with Kanye Wes...,NaN,0.000000,0.000000,0.333333,0.181310


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [229]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [230]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [231]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [232]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [233]:
print(response["answer"])

Taylor Swift is feuding with Kim Kardashian.


In [234]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [235]:
print(response["answer"])

Taylor Swift and Kim Kardashian's feud started escalating in 2016 when Kanye West released a song called "Famous" that made jabs at Taylor Swift. The tension increased with West's controversial rant during his appearance on Saturday Night Live. The feud seems to stem from a combination of personal and professional conflicts between the parties involved.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [236]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [237]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [238]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/90 [00:00<?, ?it/s]

In [239]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What was the response on the internet when #Ki...,The Internet immediately seemed to side with t...,"[actual lyrics, 'I made that bitch famous.'""\n...",The Internet immediately seemed to side with t...,1.000000,0.808803,1.00,1.000000,0.997215
1,What is the significance of Taylor Swift's vau...,The significance of Taylor Swift's vault track...,[Vault Track 'Is It Over Now?': Listen\nBy Dan...,nan,1.000000,1.000000,0.00,0.000000,0.181551
2,How did Taylor Swift use Instagram videos to r...,Taylor Swift used a series of mysterious Insta...,[with a series of mysterious Instagram videos ...,Taylor Swift used Instagram videos depicting a...,1.000000,0.964812,1.00,0.916667,0.618408
3,What organization did Taylor Swift mention as ...,Taylor Swift mentioned the World Health Organi...,"[me in donating during this crisis,"" continued...",The World Health Organization and Feeding America,1.000000,0.981268,1.00,0.833333,0.967923
4,What was the significance of Kim Kardashian re...,Kim Kardashian released video footage of Kanye...,"[March 20, 2020: The full phone call\nbetween ...",nan,1.000000,0.971646,0.20,0.500000,0.176102
5,What is the timeline of Taylor Swift and Kim K...,The timeline of Taylor Swift and Kim Kardashia...,"[4/23/24, 6:05 PM\nA Timeline of Taylor Swift ...",nan,1.000000,0.991756,0.00,0.000000,0.176835
6,What does Taylor Swift hope to receive from pe...,Taylor Swift hopes to receive an apology from ...,[11/22\nPHOTO: KEVIN WINTER/GETTY\nIn a candid...,It would be nice if we could get an apology fr...,1.000000,0.907966,1.00,1.000000,0.721115
7,What was the response on the internet when #Ki...,The Internet immediately seemed to side with t...,"[actual lyrics, 'I made that bitch famous.'""\n...",The Internet immediately seemed to side with t...,1.000000,0.808803,1.00,1.000000,0.997214
8,"What impact did the feud between Taylor Swift,...","The feud between Taylor Swift, Kanye West, and...",[19/22\nMUSIC\nTaylor Swift Says Feud with Kan...,"The feud between Taylor Swift, Kanye West, and...",1.000000,0.920179,1.00,1.000000,0.665559
9,How did Taylor Swift address the controversy w...,Taylor Swift addressed the controversy with Ka...,[asked her to release the song on her Twitter ...,Swift addressed the controversy with Kanye Wes...,0.500000,1.000000,0.00,0.588889,0.604446


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [240]:
results

{'faithfulness': 0.9643, 'answer_relevancy': 0.6933, 'context_recall': 0.6296, 'context_precision': 0.6852, 'answer_correctness': 0.5710}

And see how our advanced retrieval modified our chain!

In [241]:
advanced_retrieval_results

{'faithfulness': 0.9352, 'answer_relevancy': 0.9121, 'context_recall': 0.6778, 'context_precision': 0.6327, 'answer_correctness': 0.6470}

In [242]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.964286,0.935185,-0.029101
1,answer_relevancy,0.693310,0.912108,0.218798
2,context_recall,0.629630,0.677778,0.048148
3,context_precision,0.685185,0.632720,-0.052465
4,answer_correctness,0.570997,0.646980,0.075983


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [263]:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [264]:
vector_store = qdrant = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="Taylor Swift - Fued - MQR",
)

In [265]:
new_retriever = vector_store.as_retriever()

In [266]:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [267]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [ ]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [ ]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [260]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/90 [00:00<?, ?it/s]

In [261]:
new_advanced_retrieval_results

{'faithfulness': 0.9062, 'answer_relevancy': 0.8735, 'context_recall': 0.5278, 'context_precision': 0.6599, 'answer_correctness': 0.5463}

In [262]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.964286,0.935185,0.906250,-0.028935,-0.058036
1,answer_relevancy,0.693310,0.912108,0.873517,-0.038591,0.180207
2,context_recall,0.629630,0.677778,0.527778,-0.150000,-0.101852
3,context_precision,0.685185,0.632720,0.659921,0.027200,-0.025265
4,answer_correctness,0.570997,0.646980,0.546277,-0.100703,-0.024720


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [253]:
### YOUR CODE HERE